In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/HandWritten/HandwrittenDataset

/content/drive/MyDrive/HandWritten/HandwrittenDataset


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16

def create_dataframe_digits(data_path):
    class_categories = {
        '0': 0,
        '1': 1,
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
    }
    image_paths = []
    image_outputs = []
    for category, output in class_categories.items():
        folder_path = os.path.join(data_path, category)      # data_path/category/
        for image in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image)
            image_paths.append(image_path)
            image_outputs.append(output)
    df = pd.DataFrame({
        'path': image_paths,
        'output': image_outputs
    })
    return df

DATA_PATH='/content/drive/MyDrive/HandWritten/HandwrittenDataset'
train_df=create_dataframe_digits(os.path.join(DATA_PATH,'digits_train_set'))
test_df=create_dataframe_digits(os.path.join(DATA_PATH,'digits_test_set'))

IMG_DIM=128
def imread(img_path):
    img = cv2.imread(img_path)
    img = img.astype('float32')/255.0
    img = cv2.resize(img, (IMG_DIM, IMG_DIM))
    return img

def read_images(img_paths):
    n = len(img_paths)
    X = np.zeros((n, IMG_DIM, IMG_DIM, 3), dtype='float32')
    for i in tqdm(range(n)):
        path = img_paths[i]
        img = imread(path)
        X[i] = img
    return X

x_train=read_images(train_df['path'])
y_train=train_df['output']
x_test=read_images(test_df['path'])
y_test=test_df['output']

# Load the pre-trained VGG16 model
vgg16 = VGG16(include_top=False, input_shape=(IMG_DIM, IMG_DIM, 3), pooling='avg')

# Freeze the weights of the VGG16 layers
for layer in vgg16.layers:
    layer.trainable = False

# Create a new model that includes the VGG16 layers followed by some additional layers
model = tf.keras.models.Sequential()
model.add(vgg16)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

# Save the model
model.save('handwritten_transfer.model')

# Predict on the test set
y_pred = model.predict(x_test)

# Define a dictionary to map from class indices to class labels
code = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}

# Define a function to get the class label for a given class index
def getcode(n):
    return code[n]

# Visualize some test set predictions
plt.figure(figsize=(20,20))
for n, i in enumerate(list(np.random.randint(0, len(x_test), 36))):
    plt.subplot(6,6,n+1)
    plt.imshow(x_test[i])
    plt.axis('off')
    plt.title(getcode(np.argmax(y_pred[i])))

100%|██████████| 1414/1414 [03:32<00:00,  6.66it/s]


58889256/58889256 [==============================] - 1s 0us/step
Epoch 1/10
207/207 [==============================] - 1324s 6s/step - loss: 1.1243 - accuracy: 0.6591
Epoch 2/10
207/207 [==============================] - 1314s 6s/step - loss: 0.4686 - accuracy: 0.8721
Epoch 3/10
207/207 [==============================] - 1317s 6s/step - loss: 0.3325 - accuracy: 0.9054
Epoch 4/10
207/207 [==============================] - 1314s 6s/step - loss: 0.2640 - accuracy: 0.9252
Epoch 5/10
207/207 [==============================] - 1319s 6s/step - loss: 0.2306 - accuracy: 0.9348
Epoch 6/10
207/207 [==============================] - 1320s 6s/step - loss: 0.2028 - accuracy: 0.9447
Epoch 7/10
207/207 [==============================] - 1319s 6s/step - loss: 0.1847 - accuracy: 0.9469
Epoch 8/10
207/207 [==============================] - 1319s 6s/step - loss: 0.1703 - accuracy: 0.9510
Epoch 9/10
207/207 [==============================] - 1322s 6s/step - loss: 0.1565 - accuracy: 0.9556
Epoch 10/10
45/45

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-aa86d3f1b3ca>", line 92, in <cell line: 92>
    model.save('handwritten_transfer.model')
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 513, in recursive_create_dir_v2
    _pywrap_file_io.RecursivelyCreateDir(compat.path_to_bytes(path))
tensorflow.python.framework.errors_impl.FailedPreconditionError: handwritten_transfer.model; Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceba